# 使用W&B Weave监控OpenAI API<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" /><!--- @wandbcode{weave_openai_client_qs} --><a target="_blank" href="https://colab.research.google.com/github/wandb/weave/blob/master/examples/prompts/llm_monitoring/openai_client_quickstart.ipynb">  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在Colab中打开"/></a>**注意：** 运行此Colab需要一个[OpenAI API密钥](https://platform.openai.com/account/api-keys)。使用W&B OpenAI集成来监控OpenAI API调用，并了解您的项目和团队如何利用LLMs。在这个示例中，我们将生成模板化的Weave Boards：LLM使用监控仪表板，您可以从UI中探索和自定义。* 自动跟踪LLM使用情况，并汇总有用的指标，如成本、延迟和吞吐量，跨您的项目/团队* 动态查询并从所有OpenAI API调用的日志中获取见解* 通过可视化迭代，对数据进行切片、聚合和探索；自定义面板以关注有趣的模式；通过交互式仪表板更轻松地与团队分享进展<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/assets/full_board_view.png">[体验此Weave Board的实时版本 →](http://wandb.me/llm-monitoring-board)#### 初次接触Weights & Biases？[-> 在这里注册账户 <-](https://wandb.ai/site)# 步骤0：设置安装依赖项，登录到W&B，以便保存和分享您的工作，并使用OpenAI进行身份验证。

In [ ]:
# 如果尚未安装!pip install -qqq weave openai tiktoken wandb

In [ ]:
import wandbwandb.login()

In [ ]:
import weaveimport osWANDB_BASE_URL = "https://api.wandb.ai"os.environ["WANDB_BASE_URL"] = WANDB_BASE_URL

In [ ]:
# 使用OpenAI进行身份验证from getpass import getpassif os.getenv("OPENAI_API_KEY") is None:  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"print("OpenAI API key configured")

# 步骤 1：配置W&B中的数据流和存储将WB_ENTITY设置为您的wandb用户名或团队名称。登录到W&B并转到主页[wandb.ai/home](https://wandb.ai/home) ，在左侧边栏的“个人资料”和“团队”下查看有效选项。

In [ ]:
WB_ENTITY = "" # 设置为您的WandB用户名或团队名称WB_PROJECT = "weave" # 本作品的顶级目录STREAM_NAME = "openai_logs" # 记录表，用于存储随着OpenAI API调用实时流入的日志数据。

# 第2步：调用init_monitor()要开始监控OpenAI API的使用情况，请调用`init_monitor(<stream>)`，其中`<stream>`的格式为`<wandb_team_or_user>/<wandb_project>/<stream_name>`。该流记录并存储所有OpenAI API调用。运行此单元格将打印出一个链接，以查看Weave UI中的当前项目。

In [ ]:
from weave.monitoring import openai, init_monitorm = init_monitor(f"{WB_ENTITY}/{WB_PROJECT}/{STREAM_NAME}")# 为了简化，指定一个单一模型OPENAI_MODEL = 'gpt-3.5-turbo'# 预填充一些示例日志r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": "hello world!"}])r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": "what is 2+2?"}])

# 第三步：预览监控仪表板点击上面的链接预览数据流，然后在右侧边栏中点击“OpenAI Monitor Board”以为这个数据流创建一个Weave Board。<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/assets/short_board_attempt.gif" width=75%># 第四步：探索和了解您的LLM使用情况要保存您的工作，请点击页面顶部自动生成的名称以重新命名该Board。要分享您的Board，请点击右上角的“Publish”。<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/assets/publish_board_short.gif" width=75%>要在迭代过程中实时可视化您的工作，您可以：* 将Board保持在单独的标签页中，并刷新以查看最新数据* 随时重命名Board以便参考，并“发布”该版本以与他人共享链接* 通过在weave.wandb.ai中导航到相关的W&B实体和W&B项目名称来查找先前保存的Boards* 或者打开一个新的Board模板实例，以从迄今为止积累的所有数据开始全新的工作接下来，我们将说明一些跟踪OpenAI API调用的方法。根据您的用例，还有许多可能性，我们迫不及待地想看到您从这些起始模板中创建的内容。# 示例## 示例0：记录提示及其完成监控一个ChatCompletion请求，并打印相应的响应，仅提取完成的文本。

In [ ]:
response = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[        {"role": "user", "content": f"What is the meaning of life, the universe, and everything?"},    ])print(response['choices'][0]['message']['content'])

## 示例1：将相关参数作为属性进行跟踪将感兴趣的参数提取出来，并将它们作为日志记录的属性进行跟踪。在这里，我们将“系统提示”与“提示模板”和“方程”参数分开进行跟踪。这次我们将打印ChatCompletion调用的完整结构化响应。

In [ ]:
system_prompt = "you always write in bullet points"prompt_template = 'solve the following equation step by step: {equation}'params = {'equation': '4 * (3 - 1)'}openai.ChatCompletion.create(model=OPENAI_MODEL,                             messages=[                                    {"role": "system", "content": system_prompt},                                    {"role": "user", "content": prompt_template.format(**params)},                                ],                             # 您可以向记录的日志中添加额外的属性。                             # 请参阅 monitor_api 笔记本以获取更多示例。                             monitor_attributes={                                 'system_prompt': system_prompt,                                 'prompt_template': prompt_template,                                 'params': params                             })

## 示例2：记录持续流的消息监控一系列消息并将结果记录为单个记录。注意：在这种格式下，不计算标记。

In [ ]:
from weave.monitoring.openai import message_from_streamr = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[        {"role": "system", "content": "You are a robot and only speak in robot, like beep bloop bop."},        {"role": "user", "content": "Tell me a 50-word story."},    ], stream=True)for s in message_from_stream(r):    print(s, end='')

## 示例3：构建提示工程实验在这里，我们将比较一些系统提示、用户问题和目标受众的玩具选项。尝试进行您自己的实验，看看当您在看板中探索并按不同参数分组时是否会出现有趣的见解。

In [ ]:
def explain_math(system_prompt, prompt_template, params):    openai.ChatCompletion.create(model=OPENAI_MODEL,                             messages=[                                    {"role": "system", "content": system_prompt},                                    {"role": "user", "content": prompt_template.format(**params)},                                ],                             # 您可以向记录的日志中添加额外的属性。                             # 请参阅 monitor_api 笔记本以获取更多示例。                             monitor_attributes={                                 'system_prompt': system_prompt,                                 'prompt_template': prompt_template,                                 'params': params                             })

In [ ]:
# 欢迎随意替换您自己的提示语 :)system_prompts = ["you're extremely flowery and poetic", "you're very direct and precise", "balance brevity with insight"]prompt_template = 'explain the solution of the following to a {audience}: {equation}'equations = ['x^2 + 4x + 9 = 0', '15 * (2 - 6) / 4']audience = ["new student", "math genius"]for system_prompt in system_prompts:    for equation in equations:        for person in audience:            params = {"equation" : equation, "audience" : person}            explain_math(system_prompt, prompt_template, params)